In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors
import matplotlib
# matplotlib.artist.getp(fig.patch) # https://matplotlib.org/stable/tutorials/intermediate/artists.html#sphx-glr-tutorials-intermediate-artists-py
import seaborn as sns
import numpy as np
import pandas as pd
from IPython.display import Math
import sympy as sym
from sympy.plotting import plot as symplot

sym.init_printing()  # automatically enable the best printer available in your environment.
sns.set()
sns.set_style("whitegrid", {'grid.linestyle': '--'})

# latex output
from matplotlib.backends.backend_pgf import FigureCanvasPgf
matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)
matplotlib.rcParams.update({
    'text.usetex': True,
    'pgf.rcfonts': False,
})
# use plt.savefig('figure.pdf') as export

In [ ]:
# usually this notebook is loaded as top-level-module and relative import of chsimpy does not work.
# so we provide the path to the chsimpy package manually
import pathlib
import sys

try:
    import chsimpy
except ImportError:
    _parentdir = pathlib.Path("./").resolve().parent
    sys.path.insert(0, str(_parentdir))
    import chsimpy
    #sys.path.remove(str(_parentdir))

# auto reload if chsimpy code changed
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
dirfiles = '../_output/v0.7-paper'
timestamp = '02032023-152812'
rawfile = f"{dirfiles}/experiment-{timestamp}-raw.csv"
sysinfofile = f"{dirfiles}/experiment-{timestamp}-metadata.csv"
aggfile = f"{dirfiles}/experiment-{timestamp}-agg.csv"
paramfile = f"{dirfiles}/solution-{timestamp}-run0.yaml"

In [ ]:
# dfagg = pd.read_csv(aggfile, index_col=0)
dfraw = pd.read_csv(rawfile, index_col=['id'])
dfraw = dfraw.drop(columns=dfraw.columns[0]) # drops unnamed column
dfagg = dfraw.loc[:, dfraw.columns != 'id'].describe()
dfagg.loc['cv'] = dfagg.loc['std'] / dfagg.loc['mean']
dfagg = dfagg.T
dfagg = dfagg.astype({"count":"int"})

sysinfo = pd.read_csv(sysinfofile, index_col='key', names=['key','value'])
solution = chsimpy.utils.yaml_load(paramfile)
count = int(dfagg['count']['A0'])
set_A0 = range(0, int(count/2))
set_A1 = range(int(count/2), count)
u0 = solution.params.XXX
ca_mean = dfagg['mean']['ca']
cb_mean = dfagg['mean']['cb']

In [ ]:
# calculate ca cb based on temperature
R = 0.0083144626181532
B = 12.81
T0 = 923.15
Ts = T0 + np.arange(-100,200,25)
count = Ts.shape[0]
x0 = np.linspace(0.7,0.825,count)
x1 = np.linspace(0.0001,0.01,count)
ca = np.zeros(count)
cb = np.zeros(count)
for i in range(0, count):
    T = Ts[i]
    A0t = 186.0575 - 0.3654*T
    A1t = 43.7207 - 0.1401*T
    cacb = chsimpy.utils.get_miscibility_gap(R=R, T=T, B=B, A0=A0t, A1=A1t,
                                             xlower=x0[i], xupper=1.0-x1[i], prec=10)
    ca[i] = cacb[0]
    cb[i] = cacb[1]
    #print(T, ca[i], cb[i])

In [ ]:
cacb = np.abs(dfraw.ca-dfraw.cb)
msize = 4
#
fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(14, 12))
#
yclabel = '$\mid c_a - c_b\mid$'
# nrow, ncol, index
axs[0,0].plot(dfraw.A0[set_A0], cacb[set_A0], '.', ms=msize)
axs[0,0].set_xlabel('$A_0$')
axs[0,0].set_ylabel(yclabel)

axs[0,1].plot(dfraw.A1[set_A1], cacb[set_A1], '.', ms=msize)
axs[0,1].set_xlabel('$A_1$')
axs[0,1].set_ylabel(yclabel)

axs[1,0].plot(dfraw.A0[set_A0], dfraw.tsep[set_A0], '.', ms=msize, c='purple')
axs[1,0].set_xlabel('$A_0$')
axs[1,0].set_ylabel('$t_{sep}$')

axs[1,1].plot(dfraw.A1[set_A1], dfraw.tsep[set_A1], '.', ms=msize, c='purple')
axs[1,1].set_xlabel('$A_1$')
axs[1,1].set_ylabel('$t_{sep}$')

axs[2,0].plot(cacb, dfraw.tsep, '.', ms=msize, c='blue')
axs[2,0].set_xlabel(yclabel)
axs[2,0].set_ylabel('$t_{sep}$')

#dfcacb = pd.melt(dfraw[['cb','ca']])
#sns.boxplot(y="variable", x="value", data=dfcacb, ax=axs[2,1])
# dfraw[['cb','ca']].boxplot(ax=axs[2,1], labels=None, positions=[1,1], vert=False)
# axs[2,1].set_xlabel('c_* for T = 923.15K')
# axs[2,1].set_ylabel('')
# axs[2,1].yaxis.set_ticklabels([])
#axs[2,1].plot(dfraw.A0[set_A0], dfraw.A1[set_A1], '.', ms=msize, c='blue')
#sns.kdeplot([dfraw.fac_A0[set_A0], dfraw.fac_A1[set_A1]], ax=axs[2,1])
#dfraw[['fac_A0','fac_A1']].boxplot(ax=axs[2,1], labels=None, vert=False)
a0a1=pd.DataFrame({'factor $A_0$': dfraw.fac_A0[set_A0].values, 
                   'factor $A_1$': dfraw.fac_A1[set_A1].values})
sns.boxplot(data=a0a1, ax=axs[2,1])
axs[2,1].set_xlabel('')
axs[2,1].set_ylabel('randomness')
fig.subplots_adjust(top=0.95)
fig.suptitle(f"Statistics of the simulation with {count} random $A_0$, $A_1$ (1\% range)")

plt.savefig('tex/fig01.pdf')
display(fig)
matplotlib.pyplot.close()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(8, 5), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
#
# nrow, ncol, index
sns.kdeplot(dfraw.tsep, ax=axs[0])
axs[0].set_xlabel('')
axs[0].set_ylabel('dist')

sns.boxplot(dfraw.tsep, orient='h', ax=axs[1])
axs[1].set_xlabel('time (in s)')
axs[1].set_ylabel('')
axs[1].set(yticklabels=[])
# convert iterations to time-scale on x-axis, https://stackoverflow.com/a/74544761
plt.xticks(ticks=plt.xticks()[0][1:], 
           labels=np.round(solution.time_fac * np.array(plt.xticks()[0][1:], dtype=np.float64)))
fig.suptitle(f"Density and boxplot of $t_{{sep}}$ with {count} random $A_0$, $A_1$ (1\% range)")

plt.savefig('tex/fig02.pdf')
display(fig)
matplotlib.pyplot.close()

In [ ]:
yoffset = 1.005
# temperature ca cb points
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 5))
#
# nrow, ncol, index
axs.plot((ca,cb), (Ts,Ts), 'o', ms=4, color='0.5')
axs.axvline(x=ca_mean, ls='dashed', lw=0.8, color='#557799')
axs.axvline(x=cb_mean, ls='dashed', lw=0.8, color='#557799')
axs.axhline(T0, ls='dashed', lw=0.8, color='#557799')
axs.plot((u0), (T0), 'o', ms=5, color='red')
axs.set_xlim((0.7, 1.0))

ax2 = axs.twinx()
dfraw[['cb','ca']].boxplot(ax=ax2, labels=None, positions=[T0, T0], vert=False, widths=(30, 30), showfliers=True)
ax2.set_ylabel('')
ax2.set_yticks(axs.get_yticks())
ax2.set_ylim(axs.get_ylim())
#ax2.yaxis.set_ticklabels([])

axs.set_xlabel('mole fraction (SiO$_2$)')
axs.set_ylabel('temperature (in K)')
axs.text(u0, yoffset*T0, f"$u_0$ (${round(100*u0)}\%$ SiO$_2$)",
         usetex = True,
         horizontalalignment='center',
         verticalalignment='bottom')
axs.text(0.71, yoffset*T0, f"{T0} K")
axs.set_title('Theoretical and simulated miscibility gaps')

plt.savefig('tex/fig03.pdf')
display(fig)
matplotlib.pyplot.close()

In [ ]:
R = 0.0083144626181532
B = 12.81
T = 923.15
A0 = -151.26151
A1 = -85.612615

jitter_Arellow = 0.995
jitter_Arelhigh = 1.005
runs = 100

rng = np.random.default_rng(2023)
randv = rng.uniform(jitter_Arellow, jitter_Arelhigh, size=(2, runs))
seta0 = A0 * randv[0]
seta1 = A1 * randv[1]

In [ ]:
x = sym.Symbol('x', real=True)
c = x
E = (R*T * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
m = (E.subs(x, cb_mean) - E.subs(x, ca_mean))/(cb_mean-ca_mean)

p1 = symplot(E, (x, 0.7, 0.999), show=False, 
             line_color='black', 
             backend='matplotlib',
             size=(8, 6),
             axis_center=[0.685, -98.65],
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole',
             title=f"Estimated Gibbs free energies with {runs} random $A_0$, $A_1$ (1\% range)")
for i in range(0, runs):
    A0 = seta0[i]
    A1 = seta1[i]
    Etmp = (R*T * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
    ptmp = symplot(Etmp, (x, 0.7, 0.999), show=False, line_color='0.8')
    p1.append(ptmp[0])
p1.append(p1[0]) # again, just for overdrawing
# tangent for p1
p2 = symplot(m*(x-ca_mean)+E.subs(x, ca_mean), (x, 0.7, 0.999), show=False, line_color='blue')
p1.append(p2[0])
p1back = p1.backend(p1)
p1back.fig.gca().axvline(x=ca_mean, ls='dashed', lw=0.8, color='#557799')
p1back.fig.gca().axvline(x=cb_mean, ls='dashed', lw=0.8, color='#557799')

p1back.process_series()
p1back.fig.savefig('tex/fig04.pdf')
p1back.show()

In [ ]:
R = 0.0083144626181532
B = 12.81
T0 = 923.15
Ts = [T0, T0+150]
A0 = -151.26151
A1 = -85.612615
c = x
E = (R*Ts[0] * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))

p1 = symplot(E, (x, 0.7, 0.999), show=False, 
             line_color='black', 
             backend='matplotlib',
             axis_center=[0.69, -98.5],
             title=f"T={Ts[0]}",
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole')

A0 = 186.0575 - 0.3654*Ts[1]
A1 = 43.7207 - 0.1401*Ts[1]
Etmp = (R*Ts[1] * (c*(sym.log(c) - B) + (1-c)*sym.log(1-c)) + (A0 + A1*(1-2*c)) * c * (1-c))
p2 = symplot(Etmp, (x, 0.7, 0.999), show=False, 
             line_color='0.2',
             axis_center=[0.69, -120.0],
             title=f"T={Ts[1]}",
             xlabel='mole fraction (SiO_2)', ylabel='energy per mole'
            )

plotgrid = sym.plotting.PlotGrid(1, 2, p1, p2, show=False, size=(10, 5))
plotgrid.show()

In [ ]:
R = 0.0083144626181532
B = 12.81
T0 = 923.15
T = T0+150
A0 = -151.26151
A1 = -85.612615
chsimpy.utils.get_miscibility_gap(R=R, T=T, B=B, A0=A0, A1=A1,
                                  xlower=0.85, xupper=0.99, prec=10)

In [ ]:
s = dfagg.style
s = s.format(precision=3)
xagg = s.to_latex(hrules=True)
xagg = xagg.replace('%','\%')
xagg = xagg.replace('fac_','fac')
content = r'\documentclass{scrartcl}\usepackage{graphicx,booktabs}'
content +=r'\begin{document}'
content += xagg
content += r'\includegraphics[width=\textwidth]{fig01.pdf}\newline'
content += r'\includegraphics[width=\textwidth]{fig02.pdf}\newline'
content += r'\includegraphics[width=\textwidth]{fig03.pdf}\newline'
content += r'\includegraphics[width=\textwidth]{fig04.pdf}\newline'
content += r'\end{document}'

with open('tex/fig.tex','w') as f:
    f.write(content)